In [2]:
from pyspark.sql import SparkSession, Window
import pyspark.sql.functions as F

In [3]:
spark: SparkSession = SparkSession.builder.appName("teste_itau").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/19 13:24:21 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
df_clients = spark.read.csv("../datasets/clientes_sinteticos.csv", header=True, sep=",")

In [5]:
df_clients.show()

+-----------+-----------------+--------------------+-------------------+-----------------+----------------+--------------------+---------------------+--------------+---------+--------+
|cod_cliente|       nm_cliente|     nm_pais_cliente|  nm_cidade_cliente|   nm_rua_cliente|num_casa_cliente|    telefone_cliente|dt_nascimento_cliente|dt_atualizacao|tp_pessoa|vl_renda|
+-----------+-----------------+--------------------+-------------------+-----------------+----------------+--------------------+---------------------+--------------+---------+--------+
|        980|    Krista Rogers|          Guadeloupe|          Kellyview|      Barker Walk|            3309|      (95)39194-2483|           2001-10-29|    2024-04-18|       PJ|57900.19|
|        177| Veronica Russell|       Guinea-Bissau|          Romanside|    Tucker Canyon|             268|      (42)71167-9960|           1991-08-03|    2024-10-28|       PJ|78775.44|
|        267|   William Hughes|            Mongolia|         Taylorview|   

In [6]:
w_rank = Window.orderBy(F.col("up_count").desc())

df_top_updates = (
    df_clients.groupBy("cod_cliente")
    .agg(F.count("dt_atualizacao").alias("up_count"))
    .withColumn("rank", F.dense_rank().over(w_rank))
    .filter(F.col("rank") <= 5)
)

In [7]:
df_top_updates.show(100)

25/04/19 13:24:24 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/04/19 13:24:24 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/04/19 13:24:25 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/04/19 13:24:25 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/04/19 13:24:25 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/04/19 13:24:25 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/04/19 1

+-----------+--------+----+
|cod_cliente|up_count|rank|
+-----------+--------+----+
|        878|       5|   1|
|        479|       5|   1|
|        396|       5|   1|
|        855|       4|   2|
|        663|       3|   3|
|         71|       3|   3|
|        980|       3|   3|
|        850|       3|   3|
|        503|       3|   3|
|        632|       3|   3|
|        502|       3|   3|
|        227|       3|   3|
|        620|       3|   3|
|        177|       3|   3|
|        464|       3|   3|
|         93|       3|   3|
|        516|       3|   3|
|        127|       3|   3|
|        925|       3|   3|
|        972|       3|   3|
|        296|       2|   4|
|        334|       2|   4|
|        975|       2|   4|
|        625|       2|   4|
|        686|       2|   4|
|        970|       2|   4|
|        570|       2|   4|
|        423|       2|   4|
|        813|       2|   4|
|        712|       2|   4|
|        267|       2|   4|
|        520|       2|   4|
|        908|       

In [ ]:
df_ages_avg = df_clients.withColumn(
    "nu_idade_cliente",
    F.round(F.datediff(F.current_date(), F.col("dt_nascimento_cliente")) / 365.25),
).select(F.round(F.avg("nu_idade_cliente")).cast("int").alias("nu_media_idade_cliente"))

In [26]:
df_ages_avg.show()

+----------------------+
|nu_media_idade_cliente|
+----------------------+
|                    50|
+----------------------+

